In [ ]:
در مساله باز شدن پوزیشن باید خیلی دقت کنم که اگه یه مقدار از زمان پوزیشن گذشت و باز نکرد بی خیال بشم یا حداقل قیمتی که باز میشه را سیو کنم تا بتونم بعدا براساس همون 
قیمت کنترل بهتر داشته باشم

# Get Account Overview

In [3]:
import requests
import json
import hmac
import hashlib
import base64
from urllib.parse import urlencode
import time
import uuid
# api_key = '62dff07d37a609000198b667'
# api_secret = '37796955-d913-4499-9905-91327119ba3c'
# api_passphrase = '09150286678'
# base_uri = 'https://api-sandbox-futures.kucoin.com'


api_key = '62e3e694054f1e0001127930'
api_secret = 'a75605e8-aea9-4eb3-bc2a-d90c0c063a3f'
api_passphrase = '92237302790'
base_uri = 'https://api-futures.kucoin.com'

def get_headers(method, endpoint):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method + endpoint
    signature = base64.b64encode(hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest()).decode()
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest()).decode()
    return {   "KC-API-SIGN": signature,
                "KC-API-TIMESTAMP": str(now),
                "KC-API-KEY": api_key,
                "KC-API-PASSPHRASE": passphrase,
                "KC-API-KEY-VERSION": "2",
            "Content-Type": "application/json"}

getmethod={'GET':{'1':'deposits','method':'GET'}}


method = getmethod['GET']['method']
endpoint = '/api/v1/%s' % (getmethod['GET']['1'])
response = requests.request(method, base_uri+endpoint, headers=get_headers(method,endpoint))
print(response.status_code)
print(response.json())

404
{'code': '404', 'msg': 'Not Found', 'retry': False, 'success': False}


# place order

In [ ]:
import requests
import json
import hmac
import hashlib
import base64
from urllib.parse import urlencode
import time
import uuid
import datetime
import threading
from websocket import create_connection
from ta_calculat import runmain
import pandas as pd
import re
import chime
import traceback
import numpy as np
threads=[]
runpricetr=1
ws=0
alive_positions=pd.DataFrame()
detail_accuonts=pd.DataFrame()
base_uri = 'https://api-futures.kucoin.com'
def creat_accuont(symbol):
    global detail_accuonts
    def accuo():
        # api_key = '62dff07d37a609000198b667'
        # api_secret = '37796955-d913-4499-9905-91327119ba3c'
        # api_passphrase = '09150286678'
        # base_uri = 'https://api-sandbox-futures.kucoin.com'
        # اگر یک کاربر مشخصات ای پی آی اون تغییر کرد باید راهی بزاریم که بگه اون کاربر قبلی هست و در ازای ای پی آی جدید یک مقدار کم بگیریم
        # معمولا ای پی آی پسفرض تغییر نمیکند و به عنوان تشخیص کاربر بزاریم

        api_key = '62e3e694054f1e0001127930'
        api_secret = 'a75605e8-aea9-4eb3-bc2a-d90c0c063a3f'
        api_passphrase = '92237302790'
        base_uri = 'https://api-futures.kucoin.com'
        user=[{'api_key':'62e3e694054f1e0001127930' , 'api_secret' : 'a75605e8-aea9-4eb3-bc2a-d90c0c063a3f',
        'api_passphrase' : '92237302790'}]


        ind=['availableBalance','calculated_size','type_ballance','api_key','api_passphrase','stop_trail',
             'api_secret','id','leverage','side','size_position','symbol','status','stopPrice_sl',
             'trailprice','actionprice','sl','st','status_sl','orderId_position','orderId_sl','last_time']
        column=[]
        for i in range(len(user)):
            column.append('user%s' % (i+1))
        detail_accuonts=pd.DataFrame(columns=column,index=ind)
        for i in range(len(user)):
            detail_accuonts.iloc[ind.index('api_key'),column.index('user%s' % (i+1))]=user[i]['api_key']
            detail_accuonts.iloc[ind.index('api_secret'),column.index('user%s' % (i+1))]=user[i]['api_secret']
            detail_accuonts.iloc[ind.index('api_passphrase'),column.index('user%s' % (i+1))]=user[i]['api_passphrase']
            detail_accuonts.iloc[ind.index('status'),column.index('user%s' % (i+1))]=0
            detail_accuonts.iloc[ind.index('status_sl'),column.index('user%s' % (i+1))]=0
            detail_accuonts.iloc[ind.index('type_ballance'),column.index('user%s' % (i+1))]=['XBT','USDT','XRP','ETH','DOT']
        detail_accuonts.loc['last_time',:]=datetime.datetime.utcnow()
        return detail_accuonts
    
    try:
        detail_accuonts=pd.read_pickle('detail_accuonts.pkl')
        print('exist position')
        multi_threading(func=check_exist_alive_position,accuonts=detail_accuonts.columns)
#         t=datetime.datetime.utcnow()
#         for i in detail_accuonts.columns:
#             minute=((t-detail_accuonts.loc['last_time',i])/60).seconds
#             if minute>60:
#                 detail_accuonts=accuo()
#                 response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=i)
#                 if len(response['data'])!=0:
#                     print('you have open position')
#                     time.sleep(2000)
    except Exception as error:
        detail_accuonts=accuo()
        

                
    return detail_accuonts

def runprice(symbol):
    global price
    global rond
    global runpricetr
    global ws
    url='https://api.kucoin.com/api/v1/bullet-public'
    token=json.loads(requests.post(url).text)['data']['token']
    ws=create_connection('wss://ws-api.kucoin.com/endpoint?token=%s&[connectId=%s]' % (token,uuid.uuid1()))

    msg = {"id":"%s" % (uuid.uuid1()),
        "type": "subscribe",
        "topic": "/contractMarket/ticker:%s" %(symbol),
        "response": True}
    msg=json.dumps(msg)
    ws.send(msg)
    i=0
    runpricetr=1
    while runpricetr==1:
        try:
            p = json.loads(ws.recv())#
            if "data" in p:
                if 'price' in p['data']:
                    price=p['data']['price']
#                     rond=len(str(price).split(".")[1])
                    print(price,end='-')

            if i==3:
                msg = {"id":"%s" % (uuid.uuid1()),
            "type":"pong"
                      }
                msg=json.dumps(msg)
                ws.send(msg)
                i=0
            i+=1
        except Exception as error:
#             print('break connection with websocket')
            print('.',end='-')
            thread=threading.Thread(target=runprice,args=(symbol,))
            thread.start()
            break

def refresh_runprice(symbol) :
    global price
    global runpricetr
    global ws
    pr=0
    prr=1
    x=1
    while x==1:
        pr=price    
        time.sleep(5)
        prr=price
        if pr==prr:
            ws.close()
            runpricetr=0
            thread=threading.Thread(target=runprice,args=(symbol,))
            thread.start()
    return

def time_sleep(timeframe):
    t=(datetime.datetime.utcnow() + datetime.timedelta(hours=+8)).strftime('%H:%M:%S.%f')
    tm=int(t[3:5])
    ts=int(t[6:8])
    for i in range(timeframe+1):
        if tm%timeframe==0:
            i=timeframe
            sleep=(i-1)*60 +(60-ts+5)
            time.sleep(sleep)
            break
        if (tm+i)%timeframe==0:
            sleep=(i-1)*60 +(60-ts+5)
            time.sleep(sleep)
            break
    return

def set_param_orders(leverage,side,size,stopPrice_sl=None,stopPrice_tp=None,maintrade=True,
                     symbol=None,sl_trade=True,tp_trade=True,close_position=False):
    listparams=[]  #   for main position  listparams=['main position','stop los', 'take profit']
    if maintrade==True:
        params ={'clientOid': str(uuid.uuid1()),
                'side': side,
                'symbol': symbol,
                'type': "market",
                'size': str(size),
                'leverage': str(leverage),
                'remark':'maintrade'}
        data_json = json.dumps(params)
        listparams.append(data_json)
    
    if side=='buy':

        if sl_trade==True:
            params ={'symbol':symbol,
                    'side':'sell',
                    'leverage':str(leverage),
                    'type': 'market',
                    'clientOid':str(uuid.uuid1()),
                    'size':str(size),
                    'stop':'down',
                    'stopPrice':str(stopPrice_sl),
                    'stopPriceType':'MP',
                    'closeOrder':'true'}
            data_json = json.dumps(params)
            listparams.append(data_json)

        if tp_trade==True:
            params ={'symbol':symbol,
                    'side':'sell',
                    'leverage':str(leverage),
                    'type': 'market',
                    'clientOid':str(uuid.uuid1()),
                    'size':str(size),
                    'stop':'up',
                    'stopPrice':str(stopPrice_tp),
                    'stopPriceType':'MP',
                    'closeOrder':'true'}
            data_json = json.dumps(params)
            listparams.append(data_json)
        
        if close_position==True:
            params ={'symbol':symbol,
                    'side':'sell',
                    'leverage':str(leverage),
                    'type': 'market',
                    'clientOid':str(uuid.uuid1()),
                    'size':str(size),
                    'stopPriceType':'MP',
                    'closeOrder':'true'}
            data_json = json.dumps(params)
            listparams.append(data_json)
    
    elif side=='sell':

        if sl_trade==True:
            params ={'symbol':symbol,
                    'side':'buy',
                    'leverage':str(leverage),
                    'type': 'market',
                    'clientOid':str(uuid.uuid1()),
                    'size':str(size),
                    'stop':'up',
                    'stopPrice':str(stopPrice_sl),
                    'stopPriceType':'MP',
                    'closeOrder':'true'}
            data_json = json.dumps(params)
            listparams.append(data_json)

        if tp_trade==True:
            params ={'symbol':symbol,
                    'side':'sell',
                    'leverage':str(leverage),
                    'type': 'market',
                    'clientOid':str(uuid.uuid1()),
                    'size':str(size),
                    'stop':'down',
                    'stopPrice':str(stopPrice_tp),
                    'stopPriceType':'MP',
                    'closeOrder':'true'}
            data_json = json.dumps(params)
            listparams.append(data_json)
        
        if close_position==True:
            params ={'symbol':symbol,
                    'side':'buy',
                    'leverage':str(leverage),
                    'type': 'market',
                    'clientOid':str(uuid.uuid1()),
                    'size':str(size),
                    'stopPriceType':'MP',
                    'closeOrder':'true'}
            data_json = json.dumps(params)
            listparams.append(data_json)
        
    return listparams

def get_headers(method,endpoint,parameters,user=None):
    api_key=detail_accuonts.loc['api_key',user]
    api_secret=detail_accuonts.loc['api_secret',user]
    api_passphrase=detail_accuonts.loc['api_passphrase',user]
    now = int(time.time() * 1000)
    if parameters=='':
        str_to_sign = str(now) + method + endpoint
    else:
        str_to_sign = str(now) + method + endpoint + parameters
    signature = base64.b64encode(hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest()).decode()
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest()).decode()
    return {'KC-API-SIGN': signature,
            'KC-API-TIMESTAMP': str(now),
            'KC-API-KEY': api_key,
            'KC-API-PASSPHRASE': passphrase,
            'KC-API-KEY-VERSION': '2',
            'Content-Type': 'application/json'}

def request_order(meth,num,addendpoint,base_uri=base_uri,parameters=None,user=None):
    global detail_accuonts
    getmethod={'GET':{'1':'account-overview?currency=','2':'transaction-history','3':'deposit-address','4':'deposit-list',
    '5':'withdrawals/quotas','6':'withdrawal-list','7':'orders?status=done','8':'stopOrders?symbol=','9':'recentDoneOrders',
    '10':'fills','11':'recentFills','12':'openOrderStatistics','13':'positions?symbol=','14':'position?symbol=',
    '15':'contracts/risk-limit/','16':'funding-history?symbol=','method':'GET'},

    'POST':{'1':'orders','2':'position/margin/auto-deposit-status','3':'position/margin/deposit-margin',
    '4':'position/risk-limit-level/change','method':'POST'},

    'DELETE':{'1':'orders/','2':'orders?symbol=','3':'stopOrders?symbol=','method':'DELETE' }}

    method = getmethod[meth]['method']
    if addendpoint=='':
        endpoint = '/api/v1/%s' % (getmethod[method][num])
    else:
        endpoint = '/api/v1/%s' % (getmethod[method][num]) + addendpoint
        print(endpoint)
    re=''
    x=1
    while x==1:
        try:
            response = requests.request(method, base_uri+endpoint,
                headers=get_headers(method,endpoint,parameters,user=user), data=parameters)
            print(response.status_code)
            if response.status_code==200:
                break
            elif response.status_code==300003:
                print('no enough money')
                break
            elif response.status_code==429:
                print('Too Many Requests')
            elif response.status_code==200002:
                print('time sleep 10')
                time.sleep(11)
            elif response.status_code!=429 or response.status_code!=200002 :
                print(response.status_code ,' : ', response.json()['msg'])
                break
        except Exception as error:
            re='request_order'
            print(re)
            print(error)
            break
    
    return response.json() , re , response.status_code

def open_position(user,symbol,leverage,side,stopPrice_sl):
    # باید یک حالتی بزارم که اگه پوزیشن ها باز شد ولی استاپ گذاشته نشد باید دوباره اقدام کنه یا پیام بزاره و صدا داشته باشه چون خیلی مهم است استاپ لاس
    # باید قیمت باز شدن را برای همه پوزیشنها در نظر بگیرم چون که همه در یک قیمت باز نمیشن
    global detail_accuonts
    global price
    
    one_lot=10 #xrp
    cost=leverage*detail_accuonts.loc['availableBalance',user]
    size=int(cost/(one_lot*price))
    detail_accuonts.loc['calculated_size',user]=size


    if detail_accuonts.loc['status',user]==0:
        if side=='sell':
            print('sell-open_position')
            listparams=set_param_orders(symbol=symbol,leverage=leverage,side=side,size=size,
            stopPrice_sl=stopPrice_sl,tp_trade=False)
            for parameters in listparams:
                response,re,statusCode=request_order(num='1',parameters=parameters,meth='POST',addendpoint='',user=user)
                if listparams.index(parameters)==0:
                    if statusCode==200:
                        detail_accuonts.loc['status',user]=1
                        detail_accuonts.loc['orderId_position',user]=response['data']['orderId']
                    else:
                        break
                if listparams.index(parameters)==1:
                    if statusCode==200:
                        detail_accuonts.loc['status_sl',user]=1
                        detail_accuonts.loc['orderId_sl',user]=response['data']['orderId']

        if side=='buy':
            print('buy-open_position')
            listparams=set_param_orders(symbol=symbol,leverage=leverage,side=side,size=size,
            stopPrice_sl=stopPrice_sl,tp_trade=False)
            for parameters in listparams:
                response , re , statusCode=request_order(parameters=parameters,meth='POST',num='1',addendpoint='',user=user)
                if listparams.index(parameters)==0:
                    if statusCode==200:
                        detail_accuonts.loc['status',user]=1
                        detail_accuonts.loc['orderId_position',user]=response['data']['orderId']
                    else:
                        break
                if listparams.index(parameters)==1:
                    if statusCode==200:
                        detail_accuonts.loc['status_sl',user]=1
                        detail_accuonts.loc['orderId_sl',user]=response['data']['orderId']

    if detail_accuonts.loc['status',user]==1:
        detail_accuonts.loc['side',user]=side
        detail_accuonts.loc['size_position',user]=size
        detail_accuonts.loc['leverage',user]=leverage
        detail_accuonts.loc['symbol',user]=symbol
        x=1
        i=0
        while x==1:
#             try:
            response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=user)
            if statusCode==200 and len(response['data'])!=0:  # این باید 200 بشه و باید اینقدر تلاش کنه که جواب بگیره مگرنه باید پوزیشن را ببنده و اینو باید بعد از باز کردن پوزیشن تو قسمت اپن اردر بزارم و چک کنم که اونهایی که  اکشن پرایس و تریل پرایس ندارن بسته بشن یعنی یک تابع مخصوص کلوز بزارم
                entryprice=response['data'][0]['avgEntryPrice']
                detail_accuonts.loc['id',user]=response['data'][0]['id']
                detail_accuonts.loc['trailprice',user]=entryprice
                detail_accuonts.loc['actionprice',user]=entryprice
                break
            if  len(response['data'])==0:
                i+=1
            if i==2:
                entryprice=price
                detail_accuonts.loc['trailprice',user]=entryprice
                detail_accuonts.loc['actionprice',user]=entryprice
                break
#             except Exception as error:
#                 print(response , 'open position')
#                 print(error)
#                 traceback.print_exc()
            
    detail_accuonts.to_pickle('detail_accuonts.pkl')
                #  اگر بعد یه مقدار تکرار فرجی نشد اوردر یا پوزیشن را ببند چون ممکنه گاهی اورد تبدیل به پوزیشن نشه
    return

def control_sl(user,symbol,leverage,side):
    global detail_accuonts
    global alive_positions
    
    if detail_accuonts.loc['status',user]==1:
        parameters=''
        x=1
        c=0
        size=detail_accuonts.loc['calculated_size',user]
        if side=='buy':
            print('buy-control_sl')
            while x==1:
                response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=user)
                if c>=3:
                    listparams=set_param_orders(symbol=symbol,leverage=leverage,side=side,size=size,
                    maintrade=False,sl_trade=False,tp_trade=False,close_position=True)
                    response,re,statusCode=request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)
                    print(8)
                    if statusCode==200:
                        print(2)
                        statusCode=0
                        response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=user)

                        if (statusCode==200) and (len(response['data'])==0):
                            print(1)
                            alive_positions.drop(user, axis=1, inplace=True)
                            response,re,statusCode=request_order(parameters='',meth='DELETE',num='3',addendpoint=symbol,user=user)
                            detail_accuonts.loc['status',user]=0
                            break
                
                elif (statusCode==200) and (len(response['data'])==0):
                    print(3)
                    alive_positions.drop(user, axis=1, inplace=True)
                    request_order(parameters='',meth='DELETE',num='3',addendpoint=symbol,user=user)
                    detail_accuonts.loc['status',user]=0
                    break

                c+=1

        if side=='sell':
            print('sell-control_sl')
            while x==1:
                
                response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=user)
                if c>=3:
                    listparams=set_param_orders(symbol=symbol,leverage=leverage,side=side,size=size,
                    maintrade=False,sl_trade=False,tp_trade=False,close_position=True)
                    response,re,statusCode=request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)

                    print(7)
                    if statusCode==200:
                        print(5)
                        statusCode=0
                        response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=user)

                        if (statusCode==200) and (len(response['data'])==0):
                            print(4)
                            alive_positions.drop(user, axis=1, inplace=True)
                            response,re,statusCode=request_order(parameters='',meth='DELETE',num='3',addendpoint=symbol,user=user)
                            detail_accuonts.loc['status',user]=0
                            break

                elif (statusCode==200) and (len(response['data'])==0):
                    print(6)
                    alive_positions.drop(user, axis=1, inplace=True)
                    request_order(parameters='',meth='DELETE',num='3',addendpoint=symbol,user=user)
                    detail_accuonts.loc['status',user]=0
                    break

                c+=1
    
    return 

def control_stoptrail(user,symbol,leverage,side,stopPrice_sl,stop_trail,trailprice):
    global detail_accuonts
    global alive_positions
    global price

    print('stop_trail= ',stop_trail , '  ' ,'stopPrice_sl= ',stopPrice_sl )
    size=detail_accuonts.loc['calculated_size',user]

    if side=='buy':
        print('buy-control_stoptrail')
        x=1
        while x==1:
            listparams=set_param_orders(symbol=symbol,leverage=leverage,side=side,size=size,
            stopPrice_sl=stopPrice_sl,maintrade=False,sl_trade=True,tp_trade=False)

            response,re,statusCode=request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)
            if statusCode==200:
                detail_accuonts.loc['stopPrice_sl',user]=stopPrice_sl
                detail_accuonts.loc['trailprice',user]=trailprice
                detail_accuonts.loc['stop_trail',user]=stop_trail

    if side=='sell':
        print('sell-control_stoptrail')
        x=1
        while x==1:
            listparams=set_param_orders(symbol=symbol,leverage=leverage,side=side,size=size,
            stopPrice_sl=stopPrice_sl,maintrade=False,sl_trade=True,tp_trade=False)

            response,re,statusCode=request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)
            if statusCode==200:
                detail_accuonts.loc['stopPrice_sl',user]=stopPrice_sl
                detail_accuonts.loc['trailprice',user]=trailprice
                detail_accuonts.loc['stop_trail',user]=stop_trail
                
    alive_positions[user]=detail_accuonts[user]
    detail_accuonts.to_pickle('detail_accuonts.pkl')

def first_control(user,kwargs=None):
    global detail_accuonts
    if detail_accuonts.loc['status',user]==1:
        response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=user)
        if len(response['data'])!=0:
            response,re,statusCode=request_order(parameters='',meth='GET',num='7',addendpoint='',user=user)
            i=1
            while i < len(response['data']['items']):
                if response['data']['items'][-i]['remark']=='maintrade':
                    side=response['data']['items'][-i]['remark']
                    symbol=response['data']['items'][-i]['symbol']
                    size=respons.json()['data']['items'][-i]['size']
                    leverage=response['data']['items'][-i]['leverage']
                    break
            listparams=set_param_orders(symbol=symbol,leverage=leverage,side=side,size=size,
            maintrade=False,sl_trade=False,tp_trade=False,close_position=True)

            request_order(parameters=listparams[0],meth='POST',num='1',addendpoint=symbol,user=user)
            request_order(parameters='',meth='DELETE',num='3',addendpoint=symbol,user=user)

        if len(response['data'])==0:
            request_order(parameters='',meth='DELETE',num='3',addendpoint=symbol,user=user)
    return

def check_exist_alive_position(user,kwargs=None):
    global detail_accuonts
    x=1
    while x==1:
        response,re,statusCode=request_order(parameters='',meth='GET',num='13',addendpoint=symbol,user=user)
        if len(response['data'])==0 and statusCode==200:
            multi_threading(func=reset_users,accuonts=detail_accuonts.columns)
            request_order(parameters='',meth='DELETE',num='3',addendpoint=symbol,user=user)
            print('seccful reset detail accuont')
            break
        elif len(response['data'])!=0 and statusCode==200:
            break

def multi_threading(func,accuonts,**kwargs):
    global threads
    threads=[]
    if len(kwargs)==0:
        kwargs=None
    for i in accuonts:
        user=i
        thread=threading.Thread(target=func,args=(user,kwargs,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()
    return

def account_overview(user,kwargs=None):
    print('#####')
    global detail_accuonts
    print(detail_accuonts.loc['type_ballance',user][1])
    response,re,statusCode=request_order(parameters='',meth='GET',num='1',addendpoint=detail_accuonts.loc['type_ballance',user][1],user=user)
    detail_accuonts.loc['availableBalance',user]=response['data']['availableBalance']
    return

def reset_users(user,kwargs=None):
    print('*****')
    global detail_accuonts
    zero=['availableBalance','calculated_size','id','leverage','side','size_position',
          'symbol','status','symbol','stopPrice_sl','stop_trail','trailprice','actionprice','sl','st',
          'status_sl','orderId_position','orderId_sl']
    for i in zero:
        detail_accuonts.loc[i,user] = 0
    detail_accuonts.to_pickle('detail_accuonts.pkl')
    return

def get_signal(timeframe,limit):
    global detail_accuonts
    global alive_positions
    address='F:/ارز دیجیتال-فارکس/Python Scripts/ccxt/module/bot/آنالیز دیتا'
        
    detail={'andicator':'','signal':'','st':'','leverage':'','sl':''}
    filltred_strategy = pd.read_pickle('%s/filltred_strategy.pkl'% (address))
    order_details={}
    j=0
    for i in filltred_strategy.index:
        if j==0:
            detail['andicator']=i.split('*')[0]
            detail['signal']=i.split('*')[1]
            detail['st']=0#float(i.split('*')[2].split('-')[2])
            detail['leverage']=2#float(i.split('*')[2].split('-')[1])
            detail['sl']=0.0001#float(i.split('*')[2].split('-')[0])
            order_details[i.split('*')[0]]=detail.copy()
        j+=1
        if i.split('*')[0]!=detail['andicator']:
            detail['andicator']=i.split('*')[0]
            detail['signal']=i.split('*')[1]
            detail['st']=0#float(i.split('*')[2].split('-')[2])
            detail['leverage']=2#int(i.split('*')[2].split('-')[1])
            detail['sl']=0.0001#float(i.split('*')[2].split('-')[0])
            order_details[i.split('*')[0]]=detail.copy()
            print(order_details)
            break

    x=1
    while x==1:
        action=None
#         sellorbuy=runmain(timeframe,limit)
#         sellorbuy.to_pickle('sellorbuy.pkl')
        sellorbuy=pd.read_pickle("sellorbuy.pkl")
        for i in order_details.keys():
            print(order_details[i]['andicator'],'-',sellorbuy.loc[order_details[i]['andicator'],order_details[i]['signal']].iloc[-10::])

            if sellorbuy.loc[order_details[i]['andicator'],order_details[i]['signal']].iloc[-1,0]==0:
                print('find a signal sell', '-','type= ',sellorbuy.loc[order_details[i]['andicator'],order_details[i]['signal']].iloc[-10::])
                action=0
            elif sellorbuy.loc[order_details[i]['andicator'],order_details[i]['signal']].iloc[-1,0]==1:
                print('find a signal buy', '-','type= ',sellorbuy.loc[order_details[i]['andicator'],order_details[i]['signal']].iloc[-10::])
                action=1
            if action!=None:
                andicator=order_details[i]['andicator']
                signal=order_details[i]['signal']
                detail_accuonts.loc['st',:]=order_details[i]['st']
                detail_accuonts.loc['leverage',:]=order_details[i]['leverage']
                detail_accuonts.loc['sl',:]=order_details[i]['sl']
                detail_accuonts.to_pickle('detail_accuonts.pkl')
                chime.theme('material')
                chime.success()
                break
        if action==None:
            print('no order',end='-')
        else:
            break
        time_sleep(int(re.findall(r'\d+', timeframe)[0]))

# -----------------------------------------------------------------------
        
                # open position
    return andicator , signal , sellorbuy
    
def open_order(symbol,timeframe,limit):
    global detail_accuonts
    global alive_positions
    global price
    global commission
    global rond
    andicator , signal , sellorbuy = get_signal(timeframe,limit)
    x=1
    while x==1:
        if price==0:
            time.sleep(5)
        else:
            break
    print('\n new order\n in ' ,andicator+signal,'-' ,'type= ',sellorbuy.loc[andicator,signal].iloc[-1,0],'\n')
#     x=1
#     while x==1:
    print('action price= ',price)
    detail_accuonts.loc['symbol',:]=symbol
    p=price
    threads_op=[]
    if sellorbuy.loc[andicator,signal].iloc[-1,0]==0:
        detail_accuonts.loc['side',:]='sell'
        print('sell position')

#             for i in detail_accuonts.columns:
        detail_accuonts.loc['stopPrice_sl',:]=(1+(detail_accuonts.loc['sl',:]/detail_accuonts.loc['leverage',:]))\
        *p - p*commission
        detail_accuonts.loc['stop_trail',:]=(1-(detail_accuonts.loc['st',:]/detail_accuonts.loc['leverage',:]))\
        *p - p*commission
        detail_accuonts .loc["stopPrice_sl",:]= np.trunc(10**rond* detail_accuonts.T["stopPrice_sl"]) / 10**rond
        detail_accuonts .loc["stop_trail",:]= np.trunc(10**rond* detail_accuonts.T["stop_trail"]) / 10**rond

        for i in detail_accuonts.columns:
            thread=threading.Thread(target=open_position,args=(i,detail_accuonts.loc['symbol',i],
            detail_accuonts.loc['leverage',i],detail_accuonts.loc['side',i],detail_accuonts.loc['stopPrice_sl',i],))
            thread.start()
            threads_op.append(thread)
        for thread in threads_op:
            thread.join()
            threads_op.remove(thread)

    elif sellorbuy.loc[andicator,signal].iloc[-1,0]==1:
        detail_accuonts.loc['side',:]='buy'
        print('buy position')
#             for i in detail_accuonts.columns:
        detail_accuonts.loc['stopPrice_sl',:]=(1-(detail_accuonts.loc['sl',:]/detail_accuonts.loc['leverage',:]))\
        *p + p*commission
        detail_accuonts.loc['stop_trail',:]=(1+(detail_accuonts.loc['st',:]/detail_accuonts.loc['leverage',:]))\
        *p + p*commission
        detail_accuonts .loc["stopPrice_sl",:]= np.trunc(10**rond* detail_accuonts.T["stopPrice_sl"]) / 10**rond
        detail_accuonts .loc["stop_trail",:]= np.trunc(10**rond* detail_accuonts.T["stop_trail"]) / 10**rond

        for i in detail_accuonts.columns:
            thread=threading.Thread(target=open_position,args=(i,detail_accuonts.loc['symbol',i],
            detail_accuonts.loc['leverage',i],detail_accuonts.loc['side',i],detail_accuonts.loc['stopPrice_sl',i],))
            thread.start()
            threads_op.append(thread)
        for thread in threads_op:
            thread.join()
            threads_op.remove(thread)

#         if len(threads_op)==0:
#             break
    print('stop_trail= ',detail_accuonts.loc['stop_trail','user1'] , '  ' ,'stopPrice_sl= ',detail_accuonts.loc['stopPrice_sl','user1']  )
    return

def control_position_buy():
    global threads
    global accuonts
    global detail_accuonts
    global alive_positions
    global touched_st
    global touched_sl
    global tou_st
    global price
    global commission
    global rond
    global xloop
    x=1
    while x==1:
        detail_accuonts.loc['last_time',:]=datetime.datetime.utcnow()

        alive_positions.loc['stop_trail',:]=(1+(alive_positions.loc['st',:]/alive_positions.loc['leverage',:]))\
        *alive_positions.loc['trailprice',:] + alive_positions.loc['actionprice',:]*commission

        alive_positions .loc["stopPrice_sl",:]= np.trunc(10**rond* alive_positions.T["stopPrice_sl"]) / 10**rond
        alive_positions .loc["stop_trail",:]= np.trunc(10**rond* alive_positions.T["stop_trail"]) / 10**rond

        touched_sl=list(alive_positions.loc[:,alive_positions.loc['stopPrice_sl',:]>=price].columns).copy()
        if len(touched_sl)>0:
            print('real=',price,'  calculated_sl=',alive_positions.loc['stopPrice_sl',:])

            # باید لیست اونایی که ذخیره میشه با  اجرای مالتی تریدین هم زمان اجرا بشه که اگه هر کدومشون به لیست اضافه شد بلافاصله بری داخل مالتی تریدینگ

            print('touch sl')
            def touch_sl():
                print(10)
                global price
                global touched_sl
                global alive_positions
                t_sl=touched_sl.copy()
                x=1
                while x==1:
                    tr=list(alive_positions.loc[:,alive_positions.loc['stopPrice_sl',:]>=price].columns)
                    c=list(set(tr) ^ set(t_sl)) # پیدا کردن عضو های نامشابه
                    for i in c:
                        if i in t_sl:
                            c.remove(i)
                    touched_sl.extend(c)
                    t_sl.extend(c)
                    touched_sl=list(set(touched_sl)) # remove duplicated
                    if len(alive_positions.columns)==0:
                        print(9)
                        break
            thre=threading.Thread(target=touch_sl)
            thre.start()

            while x==1:
                if len(touched_sl)>0:
                    print(11)
                    threads_sl=[]
                    for i in touched_sl:
                        touched_sl.remove(i)
                        user=i
                        thread=threading.Thread(target=control_sl,args=(i,alive_positions.loc['symbol',i],
                        alive_positions.loc['leverage',i],alive_positions.loc['side',i],))
                        thread.start()
                        threads_sl.append(thread)
                    for thread in threads_sl:
                        thread.join()
                else:
                    print(12)
                    break

        elif len(alive_positions.columns)==0:
            print(16)
            break

        touched_st=list(alive_positions.loc[:,alive_positions.loc['stop_trail',:]<=price].columns).copy()
        tou_st=touched_st.copy()
        
        elif len(touched_st)>0:
            print('real=',price,'  calculated_st=',alive_positions.loc['stop_trail',:])

            print('touch st')
            for i in touched_st:
                alive_positions.loc['stopPrice_sl',i]=(1-(alive_positions.loc['sl',i]/alive_positions.loc['leverage',i]))\
                *price + alive_positions.loc['actionprice',i]*commission
            alive_positions .loc["stopPrice_sl",:]= np.trunc(10**rond* alive_positions.T["stopPrice_sl"]) / 10**rond
            alive_positions .loc["stop_trail",:]= np.trunc(10**rond* alive_positions.T["stop_trail"]) / 10**rond
            def touch_st():
                print(13)
                global commission
                global price
                global touched_st
                global alive_positions
                global tou_st
                global xloop
                t_st=touched_st.copy()
                xloop=1
                while xloop==1:
                    tr=list(alive_positions.loc[:,alive_positions.loc['stop_trail',:]<=price].columns)
                    c=list(set(tr) ^ set(t_st)) # پیدا کردن عضوهای نامشابه
                    for i in c:
                        if i in t_st:
                            c.remove(i)
                    touched_st.extend(c)
                    t_st.extend(c)
                    tou_st.extend(c)
                    for i in c:
                        alive_positions.loc['stopPrice_sl',i]=(1-(alive_positions.loc['sl',i]/alive_positions.loc['leverage',i]))\
                        *price + alive_positions.loc['actionprice',i]*commission
                    alive_positions .loc["stopPrice_sl",:]= np.trunc(10**rond* alive_positions.T["stopPrice_sl"]) / 10**rond
                    alive_positions .loc["stop_trail",:]= np.trunc(10**rond* alive_positions.T["stop_trail"]) / 10**rond
            thre=threading.Thread(target=touch_st)
            thre.start()

            while x==1:
                if len(touched_st)>0:
                    print(14)
                    threads_st=[]
                    for i in touched_st:
                        touched_st.remove(i)
                        user=i
                        thread=threading.Thread(target=control_stoptrail,args=(i,alive_positions.loc['symbol',i],
                                                    alive_positions.loc['leverage',i],alive_positions.loc['side',i],
                        alive_positions.loc['stopPrice_sl',i],alive_positions.loc['stop_trail',i],price,))
                        thread.start()
                        threads_st.append(thread)
                    for thread in threads_st:
                        thread.join()
                else:
                    print(15)
                    xloop=0
                    break
            # شرط گذاشتم که اگه ریسپانس 200 داشت بیاد ذخیره کنه مقدار جدید را

    return

def control_position_sell():
    global threads
    global accuonts
    global detail_accuonts
    global alive_positions
    global touched_st
    global touched_sl
    global tou_st
    global price
    global commission
    global rond
    global xloop
    x=1
    while x==1:
        detail_accuonts.loc['last_time',:]=datetime.datetime.utcnow()

        alive_positions.loc['stop_trail',:]=(1-(alive_positions.loc['st',:]/alive_positions.loc['leverage',:]))\
        *alive_positions.loc['trailprice',:] - alive_positions.loc['actionprice',:]*commission

        alive_positions .loc["stopPrice_sl",:]= np.trunc(10**rond* alive_positions.T["stopPrice_sl"]) / 10**rond
        alive_positions .loc["stop_trail",:]= np.trunc(10**rond* alive_positions.T["stop_trail"]) / 10**rond

        touched_sl=list(alive_positions.loc[:,alive_positions.loc['stopPrice_sl',:]<=price].columns).copy()

        if len(touched_sl)>0:
            print('real=',price,'  calculated_sl=',alive_positions.loc['stopPrice_sl',:])

            print('touch sl')


            def touch_sl():
                global price
                global touched_sl
                global alive_positions
                print(17)
                t_sl=touched_sl.copy()
                x=1
                while x==1:
                    tr=list(alive_positions.loc[:,alive_positions.loc['stopPrice_sl',:]<=price].columns)
                    c=list(set(tr) ^ set(t_sl)) # پیدا کردن عضو های نامشابه
                    for i in c:
                        if i in t_sl:
                            c.remove(i)
                    touched_sl.extend(c)
                    t_sl.extend(c)
                    touched_sl=list(set(touched_sl)) # remove duplicated
                    if len(alive_positions.columns)==0:
                        print(18)
                        break
            thre=threading.Thread(target=touch_sl)
            thre.start()

            while x==1:
                if len(touched_sl)>0:
                    threads_sl=[]
                    for i in touched_sl:
                        touched_sl.remove(i)
                        user=i
                        thread=threading.Thread(target=control_sl,args=(i,alive_positions.loc['symbol',i],
                                                    alive_positions.loc['leverage',i],alive_positions.loc['side',i],))
                        thread.start()
                        threads_sl.append(thread)
                    for thread in threads_sl:
                        print(21)
                        thread.join()
                
                else:
                    print(19)
                    break

        elif len(alive_positions.columns)==0:
            print(20)
            break
            
        touched_st=list(alive_positions.loc[:,alive_positions.loc['stop_trail',:]>=price].columns).copy()
#         with open('stop_trail.txt', 'w') as f:
#             f.write(json.dumps(touched_st))

        tou_st=touched_st.copy()
        elif len(touched_st)>0:
            print('real=',price,'  calculated_st=',alive_positions.loc['stop_trail',:])
            print('touch st')
            for i in touched_st:
                alive_positions.loc['stopPrice_sl',i]=(1+(alive_positions.loc['sl',i]/alive_positions.loc['leverage',i]))\
                *price - alive_positions.loc['actionprice',i]*commission
            alive_positions .loc["stopPrice_sl",:]= np.trunc(10**rond* alive_positions.T["stopPrice_sl"]) / 10**rond
            alive_positions .loc["stop_trail",:]= np.trunc(10**rond* alive_positions.T["stop_trail"]) / 10**rond
            def touch_st():
                print(22)
                global commission
                global price
                global touched_st
                global alive_positions
                global tou_st
                global xloop
                t_st=touched_st.copy()
                xloop=1
                while xloop==1:
                    tr=list(alive_positions.loc[:,alive_positions.loc['stop_trail',:]>=price].columns)
                    c=list(set(tr) ^ set(t_st)) # پیدا کردن عضوهای نامشابه
                    for i in c:
                        if i in t_st:
                            c.remove(i)
                    touched_st.extend(c)
                    t_st.extend(c)
                    tou_st.extend(c)
                    for i in c:
                        alive_positions.loc['stopPrice_sl',i]=(1+(alive_positions.loc['sl',i]/alive_positions.loc['leverage',i]))\
                        *price - alive_positions.loc['actionprice',i]*commission
                    alive_positions .loc["stopPrice_sl",:]= np.trunc(10**rond* alive_positions.T["stopPrice_sl"]) / 10**rond
                    alive_positions .loc["stop_trail",:]= np.trunc(10**rond* alive_positions.T["stop_trail"]) / 10**rond
            thre=threading.Thread(target=touch_st)
            thre.start()

            while x==1:
                if len(touched_st)>0:
                    print(24)
                    threads_st=[]
                    for i in touched_st:
                        touched_st.remove(i)
                        user=i
                        thread=threading.Thread(target=control_stoptrail,args=(i,alive_positions.loc['symbol',i],
                                                alive_positions.loc['leverage',i],alive_positions.loc['side',i],
                        alive_positions.loc['stopPrice_sl',i],alive_positions.loc['stop_trail',i],price,))
                        thread.start()
                        threads_st.append(thread)
                    for thread in threads_st:
                        thread.join()
                else:
                    print(23)
                    xloop=0
                    break
            # شرط گذاشتم که اگه ریسپانس 200 داشت بیاد ذخیره کنه مقدار جدید را
    return

def control_position(symbol,timeframe,limit):
    global price
    global detail_accuonts
    global alive_positions
    display(detail_accuonts)
    thread=threading.Thread(target=runprice,args=(symbol,))
    thread.start()

    thread=threading.Thread(target=refresh_runprice,args=(symbol,))
    thread.start()

    d=0
    x=1
    while x==1:
        if d==1:
            break

# -----------------------------------------------------------------------

        if detail_accuonts.loc['status',:].sum()==0: # if one status was 1 is not trigger
            multi_threading(func=account_overview,accuonts=detail_accuonts.columns)
            
# -----------------------------------------------------------------------
        # open position
        if detail_accuonts.loc['status',:].sum()==0:
            open_order(symbol,timeframe,limit)
# -----------------------------------------------------------------------
        #control position        
        if detail_accuonts.loc['status',:].sum()>0:
            print ('start control position')
            alive_positions=detail_accuonts.loc[:,detail_accuonts.loc['status',:]==1]
            for i in alive_positions.columns:
                if alive_positions.loc['side',i]=='buy':
                    side='buy'
                    break
                elif alive_positions.loc['side',i]=='sell':
                    side='sell'
                    break


            while x==1:
                if price==0:
                    time.sleep(5)
                else:
                    break
            if side=='buy':
                control_position_buy()

            elif side=='sell':
                control_position_sell()

# -----------------------------------------------------------------------
        multi_threading(func=check_exist_alive_position,accuonts=detail_accuonts.columns)
        d+=1

            
    return

symbol='XRPUSDTM'
timeframe='5m'
limit=540
commission=0.001
price=0
rond=4

detail_accuonts=creat_accuont(symbol)
control_position(symbol=symbol,timeframe=timeframe,limit=limit)
# مقدار لات باید خودم بهش بدم
#size is lot size
# account_overview('user1')
# creat_accuont(symbol)

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>" ))
from ta_calculat import runmain
runmain()

<Response [200]>
<Response [200]>
<Response [200]>
get ohlcv
get sell or buy


,signal0,signal1,signal2,signal3,signal4
adx,0,0,0,0,0
wma,sellorbuy 0 NaN 1 NaN 2 ...,sellorbuy 0 NaN 1 NaN 2 ...,sellorbuy 0 NaN 1 NaN 2 ...,sellorbuy 0 NaN 1 NaN 2 ...,0
macd,sellorbuy 0 NaN 1 NaN 2 ...,sellorbuy 0 NaN 1 NaN 2 ...,sellorbuy 0 NaN 1 NaN 2 ...,0,0
rsi,sellorbuy 0 NaN 1 NaN 2 ...,0,0,0,0
bbands,sellorbuy 0 NaN 1 NaN 2 ...,0,0,0,0
obv,sellorbuy 0 NaN 1 NaN 2 ...,0,0,0,0


In [11]:
import requests
import json
import hmac
import hashlib
import base64
from urllib.parse import urlencode
import time
import uuid
import pandas as pd
# api_key = '62dff07d37a609000198b667'
# api_secret = '37796955-d913-4499-9905-91327119ba3c'
# api_passphrase = '09150286678'
# base_uri = 'https://api-sandbox-futures.kucoin.com'


api_key = '6311fc9969114f0001c4af3e'
api_secret = 'c7edaa15-adef-4f5f-8f03-68b8b9cd0e20'
api_passphrase = '92237302790'
# base_uri = 'https://api-futures.kucoin.com'
base_uri='https://api.kucoin.com'
def set_param_orders():
    params = {
        'symbol':'XRPUSDTM',
        'side':'buy',
        'leverage':'1',
        'type': 'market',
        'clientOid':str(uuid.uuid1()),
        'size':'3',
        'stop':'down',
        'stopPrice':'0.3594',
        'stopPriceType':'MP',
        'remark':'Its stop-profit is set at $ 0.3635 (TP)',
    }
    data_json = json.dumps(params)
    return data_json

parameters=set_param_orders()


def get_headers(method, endpoint):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method + endpoint# + parameters
    signature = base64.b64encode(hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest()).decode()
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest()).decode()
    return {   "KC-API-SIGN": signature,
                "KC-API-TIMESTAMP": str(now),
                "KC-API-KEY": api_key,
                "KC-API-PASSPHRASE": passphrase,
                "KC-API-KEY-VERSION": "2",
            "Content-Type": "application/json"}

getmethod={'GET':{'1':'account-overview?currency=USDT','2':'transaction-history','3':'deposit-address','4':'deposit-list',
'5':'withdrawals/quotas','6':'withdrawal-list','7':'orders?status=done','8':'stopOrders?symbol=','9':'recentDoneOrders',
'10':'fills','11':'recentFills','12':'openOrderStatistics','13':'positions?symbol=','14':'position?symbol=',
'15':'contracts/risk-limit/','16':'funding-history?symbol=','17':'hist-deposits','18':'deposits','method':'GET'},
           
'POST':{'1':'orders','2':'position/margin/auto-deposit-status','3':'position/margin/deposit-margin',
'4':'position/risk-limit-level/change','5':'','6':'','method':'POST'},
           
'DELETE':{'1':'orders/','2':'orders?symbol=','3':'stopOrders?symbol=','4':'',
'5':'','6':'','method':'DELETE'}}


method = getmethod['GET']['method']
endpoint = '/api/v1/%s' % (getmethod['GET']['18'])# + 'XRPUSDTM'
response = requests.request(method, base_uri+endpoint, headers=get_headers(method,endpoint))
print(response.status_code)
# pd.DataFrame(response.json()['data']['items'])
f=response.json()
f


200


{'code': '200000',
 'data': {'currentPage': 1,
  'pageSize': 50,
  'totalNum': 1,
  'totalPage': 1,
  'items': [{'currency': 'BNB',
    'status': 'SUCCESS',
    'address': '0x262e0274c88182a01363bd9aec68d1d654510512',
    'memo': '',
    'isInner': False,
    'amount': '0.05123535',
    'fee': '0.00000000',
    'walletTxId': '0x882871039e7741a1e640adbe244690fb690470ff2aab8167c1acc83c6ddd1fbb@2951541723d523f865d283b74b770749',
    'createdAt': 1659099686000,
    'updatedAt': 1659099766000,
    'remark': 'Deposit'}]}}

In [48]:
from datetime import datetime
API_KEYs_etherscan='B28HEAXPVK6KAFM5YI72Y3TW62DPHDXV3X'
API_KEYs_bscscan='QD148ICZE32HRH3IPWNZ7N4UN8AXJAIKV5'
# u_etherscan='https://api.etherscan.io/api?module=account&action=txlist&address=0x262e0274c88182a01363bd9aec68d1d654510512&startblock=0&endblock=99999999&page=1&offset=10&sort=asc&apikey=%s' %(API_KEYs_etherscan)
u_bscscan='https://api.bscscan.com/api?module=account&action=txlist&address=0x262e0274c88182a01363bd9aec68d1d654510512&startblock=0&endblock=99999999&page=1&offset=10&sort=asc&apikey=%s' %(API_KEYs_bscscan)
response = requests.request('GET', u)
print(response.status_code)

f=response.json()
for i in range(0,2):
    print(i)
    dt_obj = datetime.fromtimestamp(int(f['result'][i]['timeStamp']))
    print("date_time:",dt_obj)
# print(f)
f=pd.DataFrame(f['result'])
f['timeStamp']=f['timeStamp'].astype(int)
f['Datetime'] = pd.to_datetime(f['timeStamp'], 
                                  unit='s')
f['datetime'] = [datetime.fromtimestamp(x/1000.0) for x in f['timeStamp']]
f
response.json()

200
0
date_time: 2022-07-29 17:30:38
1
date_time: 2022-07-29 18:21:23


{'status': '1',
 'message': 'OK',
 'result': [{'blockNumber': '19974563',
   'timeStamp': '1659099638',
   'hash': '0x882871039e7741a1e640adbe244690fb690470ff2aab8167c1acc83c6ddd1fbb',
   'nonce': '4',
   'blockHash': '0x3d94f296eabf242dce5f31e9165bbd83fc6eb72febf114c2ab33efd42b1a1688',
   'transactionIndex': '97',
   'from': '0x17feb2c0ec683f2706fdbd96ee379cb1f3c181e0',
   'to': '0x262e0274c88182a01363bd9aec68d1d654510512',
   'value': '51235350000000000',
   'gas': '21000',
   'gasPrice': '5000000000',
   'isError': '0',
   'txreceipt_status': '1',
   'input': '0x',
   'contractAddress': '',
   'cumulativeGasUsed': '11576846',
   'gasUsed': '21000',
   'confirmations': '1012834',
   'methodId': '0x',
   'functionName': ''},
  {'blockNumber': '19975566',
   'timeStamp': '1659102683',
   'hash': '0xbcf5935bad2f54523f2493eadbcb6b0a9ea8e961846432bfb89cc4135dbbe808',
   'nonce': '0',
   'blockHash': '0x414c47d4e3d1475525f8a9e3a4cc94644c975c350fde0663b346abbcd37f22ad',
   'transactionIndex

In [30]:
0x262e0274c88182a01363bd9aec68d1d654510512
0x262e0274c88182a01363bd9aec68d1d654510512

0.00015